In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py 
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn import tree 
import scipy.stats as stats
from matplotlib.colors import ListedColormap
import json

In [ ]:
#data = pd.read_csv('/content/ML_Project/datasets/train_dataset.csv')
data = pd.read_csv('datasets/train_dataset.csv')
data.head()

In [ ]:
df = data.replace([-1,'-1'],np.nan)
df['click_timestamp'].replace(['0',0],np.nan,inplace=True)
df['product_price'].replace(['0',0],np.nan,inplace=True)
print(df.info())
print("Number of Null rows : ")
print(df.isnull().sum())
print("Number of unique values : ")
print(df.nunique())

In [ ]:
df.head()

In [ ]:
col = df["product_price"]
df["pc"] = col
nan = df.isna()
nan = nan.drop(columns=['product_category(1)', 'product_category(2)',
       'product_category(3)', 'product_category(4)', 'product_category(5)',
       'product_category(6)', 'product_category(7)',"product_price",'click_timestamp','user_id','partner_id'])
nan["Sale"] = df["Sale"].apply(lambda x : not x)
fig = plt.figure(figsize=(20,10))
sns.heatmap(nan.corr(),annot=True)

In [ ]:
# plot pie charts 
def pie_plot(column,df):    
    sale = df[df["Sale"] == 1]
    not_sale = df[df["Sale"] == 0]
    fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                      subplot_titles=['sale', 'not-sale'])

    fig.add_trace(go.Pie(values  = sale[column].value_counts().values.tolist(),
                  labels  = sale[column].value_counts().keys().tolist(),
                  hoverinfo = "label+percent+name",
                  domain = dict(x = [0,.48]),
                  name = "sale customers",
                  marker_colors = [ 'orange' , 'steelblue','forestgreen' ],
                  hole = .5 ))

    fig.add_trace(go.Pie(values  = not_sale[column].value_counts().values.tolist(),
                  labels  = not_sale[column].value_counts().keys().tolist(),
                  hoverinfo = "label+percent+name",
                  domain  = dict(x = [.52,1]),                
                  name    = "not sale customers",
                  hole    = .5))

    fig.update_layout(
      plot_bgcolor  = "rgb(243,243,243)",
      paper_bgcolor = "rgb(243,243,243)",
      title_text="plot of sale and not-sale people by "+column,
      annotations=[dict(text='not-sale', x=0.75, y=0.5, font_size=20, showarrow=False),
                  dict(text='sale', x=0.25, y=0.5, font_size=20, showarrow=False)])

    py.iplot(fig)
    fig = px.pie(df, values=df[column].value_counts().values.tolist(),names=df[column].value_counts().keys().tolist())
    fig.show()
def sale_relation(col,df):
    sale = df[df["Sale"] == 1][[col,"Sale"]].astype(int)
    not_sale = df[df["Sale"] == 0][[col,"Sale"]].astype(int)
    not_sale["Sale"] = not_sale["Sale"].apply(lambda x : not x) 
    new_df = sale.groupby(col)["Sale"].sum()
    fig = plt.figure(figsize=(20,10))
    y = new_df.to_numpy()
    plt.plot(range(len(y)),y)
    plt.title(f"Sale - {col} relation")
    plt.show()
    new_df = not_sale.groupby(col)["Sale"].sum()
    y = new_df.to_numpy()
    fig = plt.figure(figsize=(20,10))
    plt.plot(range(len(y)),y)
    plt.title(f"not Sale - {col} relation")
    plt.show()
    
    
    

In [ ]:
col = df["time_delay_for_conversion"].notnull()
df["time_delay"] = col
pie_plot('time_delay',df)

In [ ]:
col = df["SalesAmountInEuro"].notnull()
df["SalesAmount"] = col
pie_plot('SalesAmount',df)

In [ ]:
col = df["product_price"].replace(0,np.nan).notnull()
df["pc"] = col
pie_plot('pc',df)

In [ ]:
# product age group 
col = df["product_age_group"]
product_age_group_list = col.unique()
product_age_group_hash_table = dict()
product_age_group_hash_table_reverse = dict()
for i,group in enumerate(product_age_group_list):
    product_age_group_hash_table[group] = i
    product_age_group_hash_table_reverse[i] = group
print(product_age_group_hash_table,product_age_group_hash_table_reverse)
col = col.apply(lambda x :product_age_group_hash_table[x]).astype("category");
df["product_age_group"] = col
pie_plot('product_age_group',df)

In [ ]:
sale_relation("product_age_group",df)

In [ ]:
# device_type
col = df["device_type"]
device_type_list = col.unique()
device_type_hash_table = dict()
device_type_hash_table_reverse = dict()
for i,group in enumerate(device_type_list):
    device_type_hash_table[group] = i
    device_type_hash_table_reverse[i] = group
print(device_type_hash_table,device_type_hash_table_reverse)
col = col.apply(lambda x :device_type_hash_table[x]).astype("category");
df["device_type"] = col
pie_plot('device_type',df)
df["device_type"].unique()

In [ ]:
sale_relation("device_type",df)

In [ ]:
# product_country
col = df["product_country"]
product_country_list = col.unique()
product_country_hash_table = dict()
product_country_hash_table_reverse = dict()
for i,group in enumerate(product_country_list):
    product_country_hash_table[group] = i
    product_country_hash_table_reverse[i] = group
print(product_country_hash_table,product_country_hash_table_reverse)
col = col.apply(lambda x :product_country_hash_table[x]).astype("category");
df["product_country"] = col
pie_plot('product_country',df)

In [ ]:
sale_relation("product_country",df)

In [ ]:
# product_gender
col = df["product_gender"]
product_gender_list = col.unique()
product_gender_hash_table = dict()
product_gender_hash_table_reverse = dict()
for i,group in enumerate(product_gender_list):
    product_gender_hash_table[group] = i
    product_gender_hash_table_reverse[i] = group
print(product_gender_hash_table,product_gender_hash_table_reverse)
col = col.apply(lambda x :product_gender_hash_table[x]).astype("category");
df["product_gender"] = col
pie_plot('product_gender',df)

In [ ]:
sale_relation("product_gender",df)

In [ ]:
#click_timestamp
col = df["click_timestamp"]
time = []
day = []
col.apply(lambda x : day.append(x.split(" ")[0]))
col.apply(lambda x : time.append(x.split(" ")[1]))
df["day"] = pd.Series(day)
df["day_time"] = pd.Series(time)
print(df["day"].unique())
print(len(df["day_time"].unique()))
pie_plot('day',df)
df["day_time_category"] = df["day_time"].apply(lambda x : int(x.split(":")[0])).astype("category")


In [ ]:
pie_plot('day_time_category',df)

In [ ]:
sale_relation('day_time_category',df)

In [ ]:
df.head()

In [ ]:
# product_brand
col = df["product_brand"]
product_brand_list = col.unique()
product_brand_hash_table = dict()
product_brand_hash_table_reverse = dict()
for i,group in enumerate(product_brand_list):
    product_brand_hash_table[group] = i
    product_brand_hash_table_reverse[i] = group
col = col.apply(lambda x :product_brand_hash_table[x]).astype("category");
df["product_brand"] = col
# product_title
col = df["product_title"]
product_title_list = col.unique()
product_title_hash_table = dict()
product_title_hash_table_reverse = dict()
for i,group in enumerate(product_title_list):
    product_title_hash_table[group] = i
    product_title_hash_table_reverse[i] = group
col = col.apply(lambda x :product_title_hash_table[x]).astype("category");
df["product_title"] = col
# audience_id
col = df["audience_id"]
audience_id_list = col.unique()
audience_id_hash_table = dict()
audience_id_hash_table_reverse = dict()
for i,group in enumerate(audience_id_list):
    audience_id_hash_table[group] = i
    audience_id_hash_table_reverse[i] = group
col = col.apply(lambda x :audience_id_hash_table[x]).astype("category");
df["audience_id"] = col
# partner_id
col = df["partner_id"]
partner_id_list = col.unique()
partner_id_hash_table = dict()
partner_id_hash_table_reverse = dict()
for i,group in enumerate(partner_id_list):
    partner_id_hash_table[group] = i
    partner_id_hash_table_reverse[i] = group
col = col.apply(lambda x :partner_id_hash_table[x]).astype("category");
df["partner_id"] = col
# user_id
col = df["user_id"]
user_id_list = col.unique()
user_id_hash_table = dict()
user_id_hash_table_reverse = dict()
for i,group in enumerate(user_id_list):
    user_id_hash_table[group] = i
    user_id_hash_table_reverse[i] = group
col = col.apply(lambda x :user_id_hash_table[x]).astype("category");
df["user_id"] = col
# product_id
col = df["product_id"]
product_id_list = col.unique()
product_id_hash_table = dict()
product_id_hash_table_reverse = dict()
for i,group in enumerate(product_id_list):
    product_id_hash_table[group] = i
    product_id_hash_table_reverse[i] = group
col = col.apply(lambda x :product_id_hash_table[x]).astype("category");
df["product_id"] = col


In [ ]:
df.head()

In [ ]:
mapping_dict = dict()
mapping_dict["product_age_group"] = [product_age_group_hash_table,product_age_group_hash_table_reverse]
mapping_dict["device_type"] = [device_type_hash_table,device_type_hash_table_reverse]
mapping_dict["product_gender"] = [product_gender_hash_table,product_gender_hash_table_reverse]
mapping_dict["product_brand"] = [product_brand_hash_table,product_brand_hash_table_reverse]
mapping_dict["product_country"] = [product_country_hash_table,product_country_hash_table_reverse]
mapping_dict["product_title"] = [product_title_hash_table,product_title_hash_table_reverse]
mapping_dict["audience_id"] = [audience_id_hash_table,audience_id_hash_table_reverse]
mapping_dict["partner_id"] = [partner_id_hash_table,partner_id_hash_table_reverse]
mapping_dict["user_id"] = [user_id_hash_table,user_id_hash_table_reverse]
mapping_dict["product_id"] = [product_id_hash_table,product_id_hash_table_reverse]

In [ ]:
df.head()

In [ ]:
cat_columns = ['product_age_group', 'device_type', 'audience_id', 'product_gender', 'product_brand'
, 'product_country', 'product_category(1)', 'product_category(2)', 'product_category(3)', 'product_category(4)', 
 'product_category(5)',  'product_category(6)']
product_category_cols = [ 'product_category(1)', 'product_category(2)', 'product_category(3)', 'product_category(4)', 
 'product_category(5)',  'product_category(6)']

In [ ]:
df[product_category_cols] = df[product_category_cols].fillna(0)

In [ ]:
df[product_category_cols].head()

In [ ]:
cat_dict = dict()
for cat in product_category_cols:
    col = df[cat]
    llist = col.unique()
    llist_hash_table = dict()
    llist_hash_table_reverse = dict()
    for i,group in enumerate(llist):
        llist_hash_table[group] = i
        llist_hash_table_reverse[i] = group
    col = col.apply(lambda x :llist_hash_table[x]).astype("category");
    df[cat] = col
    cat_dict[cat] = [llist_hash_table,llist_hash_table_reverse]
mapping_dict["category_dicts"] = cat_dict

In [ ]:
id_type = 0
class category_tree:
    def __init__(self,num):
        global id_type
        self.type = num
        self.id = id_type
        id_type = id_type+1
        self.child = dict()
    def add_child(self,childrens : list,index):
        if len(childrens) <= index:
            return 
        temp = self.child.get(childrens[index],None)
        if temp == None:
            temp = self.child[childrens[index]] = category_tree(self.type+1)
        temp.add_child(childrens,index+1)
    def get_id(self,childrens : list,index,max_depth = 7,depth_iter = 0):
        if len(childrens) <= index or depth_iter+1 > max_depth :
            return self.id
        else:
            return self.child.get(childrens[index]).get_id(childrens,index+1,max_depth,depth_iter+1)
    """
    def __dict__(self):
        temp = ""
        for key,val in self.child.items():
            temp += f'{key} : {val.__dict__}'
            print(temp)
        return f'\{\"type\": {self.type}, \"id\": {self.id}, \"child\": \{{temp}\}\}'
    """
cols = df[product_category_cols].to_numpy()
node = category_tree(0)
for row in cols:
    node.add_child(row,0)
        
            
            
        

In [ ]:
for depth in range(7):
    a = []
    for row in cols:
        a.append(node.get_id(row,0,depth))
    a = pd.Series(a)
    print(f'max_depth {depth} : path {len(a.unique())}')

In [ ]:
a = []
for row in cols:
    a.append(node.get_id(row,0))
a = pd.Series(a)
df["tree_encode"] = a

In [ ]:
df["tree_encode"].value_counts()

In [ ]:
df[product_category_cols].to_csv("utils/category_tree.csv")

In [ ]:
df[product_category_cols].head()

In [ ]:
new_col = df[product_category_cols].astype(bool)
new_col = new_col.astype(int)

In [ ]:
col = new_col["product_category(1)"]
for i in range(2,7):
    col = col+(2**(i-1))* new_col[f"product_category({i})"]

In [ ]:
df["category_encode"] = col.astype("category")
df

In [ ]:
number_of_sale =  np.count_nonzero(df.Sale.values)
number_of_not_sale = np.size(df['Sale'])-number_of_sale
sizes = (number_of_sale, number_of_not_sale)
explode = (0, 0.1)
colors =['#c2c2f0','#ffb3e6', '#c2c2f0','#ffb3e6', '#c2c2f0','#ffb3e6', '#c2c2f0','#ffb3e6']
fig1, ax1 = plt.subplots()
status = 'Sale = 1', 'Sale = 0'
ax1.pie(sizes, labels=status, autopct='%1.1f%%', colors=colors, frame=True , startangle=90, pctdistance=0.85)
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
ax1.axis('equal')  
plt.tight_layout()
plt.show()

In [ ]:
def hist_plot(column):
  fig, ax = plt.subplots(figsize=(15,8))
  sns.distplot(ax = ax,a = sale[column],hist=True, kde=True, rug=False , label = 'sale')
  sns.distplot(ax = ax ,a = not_sale[column],hist=True, kde=True, rug=False,label = 'not_sale')
  plt.legend()
  plt.show()

In [ ]:
numerical_columns = ['SalesAmountInEuro', 'time_delay_for_conversion',
       'nb_clicks_1week', 'product_price']
numeric_data_mapping = dict() # [upper bound , lowe bound , upper fill , lower fill , null fill]
df.head()

In [ ]:
def histogramX(col):
    fig = plt.figure(figsize=(20,10))
    sns.boxplot(data=pd.DataFrame(data = {"col":col}),x = "col")
    plt.show()
    fig = plt.figure(figsize=(20,10))
    sns.distplot(col, hist=True, kde=True, 
             bins=int(20), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
    plt.show()

In [ ]:
col = df["SalesAmountInEuro"]
histogramX(col)

In [ ]:
mean = col.mean()
col = col.fillna(mean)
upper = col.quantile(0.95)
lower = col.quantile(0.05)
col = col.apply(lambda x : mean if (x >upper) else lower if (x < lower) else x )
print(col.describe()) # [upper bound , lowe bound , upper fill , lower fill , null fill]
numeric_data_mapping["SalesAmountInEuro"] = [upper,lower,mean,lower,mean]
df["SalesAmountInEuro"] = col

In [ ]:
histogramX(col)

In [ ]:
col = df["time_delay_for_conversion"]
histogramX(col)

In [ ]:
mean = col.mean()
col = col.fillna(mean)
upper = col.quantile(0.95)
lower = col.quantile(0.05)
col = col.apply(lambda x : mean if (x >upper) else lower if (x < lower) else x )
print(col.describe()) # [upper bound , lowe bound , upper fill , lower fill , null fill]
numeric_data_mapping["time_delay_for_conversion"] = [upper,lower,mean,lower,mean]
df["time_delay_for_conversion"] = col

In [ ]:
histogramX(col)

In [ ]:
col = df["nb_clicks_1week"]
histogramX(col)

In [ ]:
mean = col.mean()
col = col.fillna(mean)
upper = col.quantile(0.95)
lower = col.quantile(0.05)
col = col.apply(lambda x : mean if (x >upper) else lower if (x < lower) else x )
print(col.describe()) # [upper bound , lowe bound , upper fill , lower fill , null fill]
numeric_data_mapping["nb_clicks_1week"] = [upper,lower,mean,lower,mean]
df["nb_clicks_1week"] = col

In [ ]:
histogramX(col)

In [ ]:
col = df["product_price"]
histogramX(col)

In [ ]:
pro

In [ ]:
mean = col.mean()
col = col.fillna(mean)
upper = col.quantile(0.95)
lower = col.quantile(0.05)
col = col.apply(lambda x : mean if (x >upper) else lower if (x < lower) else x )
print(col.describe()) # [upper bound , lowe bound , upper fill , lower fill , null fill]
numeric_data_mapping["product_price"] = [upper,lower,mean,lower,mean]
df["product_price"] = col

In [ ]:
histogramX(col)

In [ ]:
df = df.drop(columns = ["pc","product_category(7)","SalesAmount","time_delay"])
df.head()

In [ ]:
df.to_csv('datasets/EDA.csv')
json.dump( mapping_dict, open( "utils/mapping.json", 'w' ) )
json.dump( mapping_dict, open( "utils/numeric_data_mapping.json", 'w' ) )